In [263]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time
# from sympy import *
from scipy.linalg import schur, eigvals
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [273]:
def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)

    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # eigvx,eigvecx=la.eig(X)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    # eigvgm_,eigvecgmleft=la.eig(gmat.T)
    # eigvgm,eigvecgm=schur(gmat) # schur decomposition
    '''
    ugm,svgm,vhgm=la.svd(gmat)
    vgm=vhgm.T
    ngm=vhgm.copy()
    ngm=ngm.T 
    ngm[:,0],ngm[:,1]=svgm[0]*ngm[:,0],svgm[1]*ngm[:,1]
    '''

    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)#,ugm,svgm,vgm)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI 
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    ua,svam,vha=la.svd(Am)
    va=vha.T
    # svam=svam[0]
    # ua,va=np.zeros((N,N)),np.zeros((N,N))
    # ua[:,0]=1.0/np.sqrt(N)
    # va[:NE,0]=JE/np.sqrt(NE)
    # va[NE:,0]=-JI/np.sqrt(NI)
    # va[:,0]/=-np.linalg.norm(va[:,0])
    # svam=np.sqrt(2*(JE**2+JI**2))
    # vha=va.T

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv,ua,svam,va)
# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
  Amplit = gavgfix*typenum
  numsample = typenum**2
  Amplitg= np.zeros(numsample)
  idxc=0
  while (1):
    if idxc>=numsample:
      Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
      break
    p=np.random.random(1)
    Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
    if np.sum(Amplitg)>1.0:
      continue 
    elif np.sum(Amplitg)==1.0:
      break
    else:
      idxc +=1
      # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
  # Amplitg=0
  Amplitg*=Amplit
  Amplitg=np.sqrt(Amplitg)
  return Amplitg

# Listing all statistical properties
* ## rank-2 eigen spectrum, SVD
* ## rank-1 left $\mathbf{n}$ and right $\mathbf{m}$

In [383]:
''' RANK-1 '''
''' heterogeneous case with single zero g_{EE} '''
# strfile = '2021-01-19-11_23_52_heterogeneousNotgEE_cloudsdata'
# strfile ='2021-01-19-12_02_53_heterogeneousgII_cloudsdata'
strfile='2021-01-19-11_44_14_homogeneous_cloudsdata'
data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
meigseries,neigseries=data['meigseries'],data['neigseries']
gaverageseries=data['gaverageseries']
# # loading calculated mean and std (2*2) for rank1, 0,0 m,E 0,1 m,I
# meanloadingseries,stdloadingseries=data['meanloadingseries'],data['stdloadingseries']
# ((niter,ntrials,NE*2))
ngavg,nbatch,_=np.shape(meigseries)

# Nt=np.array([1000,1000])
# NE,NI=Nt[0],Nt[1]
# Nparams=np.array([NE,NI])
# JE,JI,a,b=1.0,3.5,-0.6,0.6
# a,b=0.0,0.0
# JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# Jparameters = np.zeros(4+1)
# Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
# Jparameters[4]=NE
nrank=1
Jparameters=data['Jparameters']
NE = int(Jparameters[4])

P = np.zeros((2,2))
P = np.array([[Jparameters[0],-Jparameters[1]],[Jparameters[2],-Jparameters[3]]])
Am=np.zeros((NE*2,NE*2))
Am[:NE,:NE],Am[:NE,NE:]=P[0,0]/NE,P[0,1]/NE
Am[NE:,:NE],Am[NE:,NE:]=P[1,0]/NE,P[1,1]/NE
# # calculate the original \bar{m} and \bar{n}
# uAm,svAm,vhAm=la.svd(Am)
# vAm=vhAm.T
# mAm,nAm=uAm[:,:nrank],vAm[:,:nrank]
# for i in range(nrank):
#     nAm[:,i]*=svAm[i]
# print('\bar{m}&\bar{n}')
# print(mAm[1::NE,0],nAm[1::NE,0])
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('\bar{m}&\bar{n}')
print(meigvecAm[1::NE,0],neigvecAm[1::NE,0])
print('eigenvalues:',eigvAm[:nrank])
# [-0.02236068 -0.02236068] [-0.04472136  0.15652476] #strfile='2021-01-19-11_44_14_homogeneous_cloudsdata'
# print(np.sum(neigvecAm[:,0]*meigvecAm[:,0]))


ar{m}ar{n}
[0.02236068+0.j 0.02236068+0.j] [ 0.04472136-0.j -0.15652476+0.j]
eigenvalues: [-2.5+0.j]


In [360]:
idxgavg=10
# generate figure
xticks = np.linspace(-4.0,1.0,5)
xlims = [-4.1,1.1]
yticks = np.linspace(-0.8,0.8,3)
ylims = [-0.81,0.81]
theta = np.linspace(0, 2 * np.pi, 200)
# fig,ax=plt.subplots(1,3,figsize=(12,4))
# ''' PLOT a sample scatter plot A '''
# ## three cases
# case one homogeneous
strfile='2021-01-19-11_44_14_homogeneous_cloudsdata'
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
nrank=1
Jparameters=data['Jparameters']
JEE,JIE,JEI,JII=Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]
Nparams=np.array([NE,NE])
# generate mean 
Am,Jsv,ua,svam,va=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# OVERALL
eigvalAm,_=la.eig(Am)
# generate X
# totally average
gaverage=gaverageseries[idxgavg]
grand = gaverage*np.ones(4)/np.sqrt(2.0)
print('pre avg:',gaverage)
## homogeneous X matrix
X,Gamp,eigvgmT,eigvecgm,gaverage=generate_localstatsmat_eig(Nparams,grand)#,ugm,svgm,vgm
print('post avg:',gaverage)
xr = gaverage*np.cos(theta)
yr = gaverage*np.sin(theta)
# mix together
Xtotal=X.copy()+Am.copy()
eigvxt,eigvecxt=la.eig(Xtotal)
# eigenvalues
meigseries,neigseries=data['meigseries'],data['neigseries']
eigvalseries=np.zeros(20)
for i in range(20):
    eigvalseries[i]=np.squeeze(neigseries[idxgavg,i,:])*np.squeeze(meigseries[idxgavg,i,:])

fig,ax=plt.subplots(3,1,figsize=(8,8))
real_eigsbulk,imag_eigsbulk=np.real(eigvxt[nrank:]),np.imag(eigvxt[nrank:])
ax[0].scatter(real_eigsbulk,imag_eigsbulk,marker='^',s=5,color='blue',alpha=0.25)
ax[0].scatter(np.real(eigvxt[:nrank]),np.imag(eigvxt[:nrank]),marker='^',s=5,color='blue',alpha=0.5)
for i in range(nrank):
    ax[0].scatter(np.real(eigvalseries),np.imag(eigvalseries),marker='^',s=5,color='blue',alpha=0.5)
xa = gaverage*np.cos(theta)
ya = gaverage*np.sin(theta)
ax[0].plot(xa, ya, color="darkred", linewidth=1.5,label=r'$r_g$')
ax[0].plot(xr, yr, color="grey", linewidth=1.5,linestyle='--',label=r'fixed $\bar{g}$')
ax[0].scatter(np.real(eigvalAm[:nrank]),np.imag(eigvalAm[:nrank]),marker='^',s=15,color='red',alpha=1.0)
ax[0].set_xlim(xlims)
ax[0].set_ylim(ylims)
ax[0].set_xticks(xticks)
ax[0].set_yticks(yticks)
ax[0].set_aspect('equal')
ax[0].legend()
ax[0].grid()

# case two heterogeneous RII
strfile ='2021-01-19-12_02_53_heterogeneousgII_cloudsdata'
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
# generate X
# max r_g
grand = np.zeros(4)
grand[3]=gaverage*np.sqrt(2) # I dominant
print('pre avg:',gaverage)
## homogeneous X matrix
X,Gamp,eigvgmT,eigvecgm,gaverage=generate_localstatsmat_eig(Nparams,grand)#,ugm,svgm,vgm
print('post avg:',gaverage)

# mix together
Xtotal=X.copy()+Am.copy()
eigvxt,eigvecxt=la.eig(Xtotal)
# eigenvalues
meigseries,neigseries=data['meigseries'],data['neigseries']
eigvalseries=np.zeros(20)
for i in range(20):
    eigvalseries[i]=np.squeeze(neigseries[idxgavg,i,:])*np.squeeze(meigseries[idxgavg,i,:])
# np.squeeze(meigseries[i,:,:NE,j]).T
real_eigsbulk,imag_eigsbulk=np.real(eigvxt[nrank:]),np.imag(eigvxt[nrank:])
ax[1].scatter(real_eigsbulk,imag_eigsbulk,marker='^',s=5,color='blue',alpha=0.25)
ax[1].scatter(np.real(eigvxt[:2]),np.imag(eigvxt[:2]),marker='^',s=5,color='blue',alpha=0.5)
for i in range(nrank):
    ax[1].scatter(np.real(eigvalseries),np.imag(eigvalseries),marker='^',s=5,color='blue',alpha=0.5)
xa = np.sqrt(2)*gaverage*np.cos(theta)
ya = np.sqrt(2)*gaverage*np.sin(theta)
ax[1].plot(xa, ya, color="darkred", linewidth=1.5,label=r'$r_g$')
ax[1].plot(xr, yr, color="grey", linewidth=1.5,linestyle='--',label=r'fixed $\bar{g}$')
ax[1].scatter(np.real(eigvalAm[:nrank]),np.imag(eigvalAm[:nrank]),marker='^',s=15,color='red',alpha=1.0)
ax[1].set_xlim(xlims)
ax[1].set_ylim(ylims)
ax[1].set_xticks(xticks)
ax[1].set_yticks(yticks)
ax[1].set_aspect('equal')
ax[0].set_aspect('equal')
ax[1].legend()
ax[1].grid()

# case two heterogeneous not gEE
strfile = '2021-01-19-11_23_52_heterogeneousNotgEE_cloudsdata'
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
# generate X
# three copy
grand = gaverage*np.ones(4)*np.sqrt(2.0/3.0)
grand[0]=0
print('pre avg:',gaverage)
## homogeneous X matrix
X,Gamp,eigvgmT,eigvecgm,gaverage=generate_localstatsmat_eig(Nparams,grand)#,ugm,svgm,vgm
print('post avg:',gaverage)

# mix together
Xtotal=X.copy()+Am.copy()
eigvxt,eigvecxt=la.eig(Xtotal)
# eigenvalues
# eigenvalues
meigseries,neigseries=data['meigseries'],data['neigseries']
eigvalseries=np.zeros(20)
for i in range(20):
    eigvalseries[i]=np.squeeze(neigseries[idxgavg,i,:])*np.squeeze(meigseries[idxgavg,i,:])

real_eigsbulk,imag_eigsbulk=np.real(eigvxt[2:]),np.imag(eigvxt[2:])
ax[2].scatter(real_eigsbulk,imag_eigsbulk,marker='^',s=5,color='blue',alpha=0.25)
ax[2].scatter(np.real(eigvxt[:nrank]),np.imag(eigvxt[:nrank]),marker='^',s=5,color='blue',alpha=0.5)
for i in range(nrank):
    ax[2].scatter(np.real(eigvalseries),np.imag(eigvalseries),marker='^',s=5,color='blue',alpha=0.5)
arg = np.sqrt((1+np.sqrt(5))/3.0)
xa = arg*gaverage*np.cos(theta)
ya = arg*gaverage*np.sin(theta)
ax[2].plot(xa, ya, color="darkred", linewidth=1.5,label=r'$r_g$')
ax[2].plot(xr, yr, color="grey", linewidth=1.5,linestyle='--',label=r'fixed $\bar{g}$')
ax[2].scatter(np.real(eigvalAm[:nrank]),np.imag(eigvalAm[:nrank]),marker='^',s=15,color='red',alpha=1.0)
ax[2].set_xlim(xlims)
ax[2].set_ylim(ylims)
ax[2].set_xticks(xticks)
ax[2].set_yticks(yticks)
ax[2].set_aspect('equal')
ax[0].set_aspect('equal')
ax[2].legend()
ax[2].grid()




pre avg: 0.5499999999999999
post avg: 0.5499999999999998
pre avg: 0.5499999999999998
post avg: 0.5499999999999998
pre avg: 0.5499999999999998
post avg: 0.5499999999999998


In [384]:
''' For rank 1 '''
# calculate \theta overlap
loadingeigvec=np.zeros((ngavg,2,nrank*2,NE,nbatch)) # ng,EI,nrank*2 mn,Npopulation,ntrials
loadingeigvecAm=np.zeros((2,nrank*2,NE))
meanloadingeigclouds,stdloadingeigclouds = np.zeros((ngavg,2,nrank*2,nbatch)),np.zeros((ngavg,2,nrank*2,nbatch))
Norm2=np.zeros((ngavg,3,nrank*2,nbatch))
Norm2Am=np.zeros((3,nrank*2,nbatch))
overlapeig,overlapeig_renorm=np.zeros((ngavg,3,nrank*2,nbatch)),np.zeros((ngavg,3,nrank*2,nbatch))
# lambda reconstruction
lambda_reconstruct=np.zeros((ngavg,3,nrank,nbatch)) # E pop, I pop, overall
for j in range(nrank):
    # eig
    loadingeigvecAm[0,j,:]=np.squeeze(meigvecAm[:NE,j]).T # m_j^E
    loadingeigvecAm[0,j+nrank,:]=np.squeeze(neigvecAm[:NE,j]).T # n_j^E
    loadingeigvecAm[1,j,:]=np.squeeze(meigvecAm[NE:,j]).T # m_j^I
    loadingeigvecAm[1,j+nrank,:]=np.squeeze(neigvecAm[NE:,j]).T # n_j^I
    Norm2Am[0,j]=np.sum(np.squeeze(loadingeigvecAm[0,j,:])**2,axis=0) # excitatory
    Norm2Am[1,j]=np.sum(np.squeeze(loadingeigvecAm[1,j,:])**2,axis=0) # inhibitory
    Norm2Am[2,j]=Norm2Am[1,j]+Norm2Am[0,j] # overall
    Norm2Am[0,j+nrank]=np.sum(np.squeeze(loadingeigvecAm[0,j+nrank,:])**2,axis=0) # excitatory
    Norm2Am[1,j+nrank]=np.sum(np.squeeze(loadingeigvecAm[1,j+nrank,:])**2,axis=0) # inhibitory
    Norm2Am[2,j+nrank]=Norm2Am[1,j+nrank]+Norm2Am[0,j+nrank] # overall
for j in range(nrank):
    for i in range(ngavg):
        # eig
        loadingeigvec[i,0,j,:,:]=np.squeeze(meigseries[i,:,:NE]).T # m_j^E
        loadingeigvec[i,0,j+nrank,:,:]=np.squeeze(neigseries[i,:,:NE]).T # n_j^E
        loadingeigvec[i,1,j,:,:]=np.squeeze(meigseries[i,:,NE:]).T # m_j^I
        loadingeigvec[i,1,j+nrank,:,:]=np.squeeze(neigseries[i,:,NE:]).T # n_j^I

        # renormalization
        # m, right eigenvectors
        Norm2[i,0,j,:]=np.sum(np.squeeze(loadingeigvec[i,0,j,:,:])**2,axis=0) # excitatory
        Norm2[i,1,j,:]=np.sum(np.squeeze(loadingeigvec[i,1,j,:,:])**2,axis=0) # inhibitory
        Norm2[i,2,j,:]=Norm2[i,1,j,:]+Norm2[i,0,j,:] # overall
        # n, left eigenvectors
        Norm2[i,0,j+nrank,:]=np.sum(np.squeeze(loadingeigvec[i,0,j+nrank,:,:])**2,axis=0) # excitatory n
        Norm2[i,1,j+nrank,:]=np.sum(np.squeeze(loadingeigvec[i,1,j+nrank,:,:])**2,axis=0) # inhibitory n
        Norm2[i,2,j+nrank,:]=Norm2[i,1,j+nrank,:]+Norm2[i,0,j+nrank,:] # overall normalization
        # overlap
        # m
        overlapeig[i,0,j,:]=np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:])) # excitatory overlap -- renormalized
        overlapeig[i,1,j,:]=np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:])) # inhibitory overlap -- renromalized
        overlapeig[i,2,j,:]=np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:]))+np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:])) # inhibitory overlap -- renormalized
        # n
        overlapeig[i,0,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))/(eigvAm[j])**2 # excitatory overlap -- renormalized
        overlapeig[i,1,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:]))/(eigvAm[j])**2# inhibitory overlap -- renormalize+
        overlapeig[i,2,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))/(eigvAm[j])**2+np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:]))/(eigvAm[j])**2# inhibitory overlap -- renormalized

        # overlap renormalizing
        # m
        overlapeig_renorm[i,0,j,:]=np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:]))/np.sqrt(Norm2[i,0,j,:])/np.sqrt(Norm2Am[0,j]) # excitatory overlap -- renormalized
        overlapeig_renorm[i,1,j,:]=np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:]))/np.sqrt(Norm2[i,1,j,:])/np.sqrt(Norm2Am[1,j]) # inhibitory overlap -- renromalized
        overlapeig_renorm[i,2,j,:]=(np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:]))+np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:])))/np.sqrt(Norm2[i,2,j,:])/np.sqrt(Norm2Am[2,j]) # overlap -- renromalized
        # n
        overlapeig_renorm[i,0,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))/np.sqrt(Norm2[i,0,j+nrank,:])/np.sqrt(Norm2Am[0,j+nrank]) # excitatory overlap -- renormalized
        overlapeig_renorm[i,1,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:]))/np.sqrt(Norm2[i,1,j+nrank,:])/np.sqrt(Norm2Am[1,j+nrank]) # inhibitory overlap -- renormalized
        overlapeig_renorm[i,2,j+nrank,:]=(np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))+np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:])))/np.sqrt(Norm2[i,2,j+nrank,:])/np.sqrt(Norm2Am[2,j+nrank])
        # statistics
        meanloadingeigclouds[i,0,j,:]=(np.mean(np.squeeze(loadingeigvec[i,0,j,:,:]),axis=0)) # m excitatory trial averaging
        meanloadingeigclouds[i,0,j+nrank,:]=(np.mean(np.squeeze(loadingeigvec[i,0,j+nrank,:,:]),axis=0)) # n excitatory trial average
        meanloadingeigclouds[i,1,j,:]=(np.mean(np.squeeze(loadingeigvec[i,1,j,:,:]),axis=0)) # m inhibitory trial average
        meanloadingeigclouds[i,1,j+nrank,:]=(np.mean(np.squeeze(loadingeigvec[i,1,j+nrank,:,:]),axis=0)) # n inhibitory trial average

        stdloadingeigclouds[i,0,j,:]=np.std(np.squeeze(loadingeigvec[i,0,j,:,:]),axis=0)
        stdloadingeigclouds[i,0,j+nrank,:]=np.std(np.squeeze(loadingeigvec[i,0,j+nrank,:,:]),axis=0)
        stdloadingeigclouds[i,1,j,:]=np.std(np.squeeze(loadingeigvec[i,1,j,:,:]),axis=0)
        stdloadingeigclouds[i,1,j+nrank,:]=np.std(np.squeeze(loadingeigvec[i,1,j+nrank,:,:]),axis=0)

        # reproduce lambda overall
        lambda_reconstruct[i,0,j,:]=np.sum(np.squeeze(meigseries[i,:,:NE])*np.squeeze(neigseries[i,:,:NE]),axis=1)
        lambda_reconstruct[i,1,j,:]=np.sum(np.squeeze(meigseries[i,:,NE:])*np.squeeze(neigseries[i,:,NE:]),axis=1)
        lambda_reconstruct[i,2,j,:]=np.sum(np.squeeze(meigseries[i,:,:])*np.squeeze(neigseries[i,:,:]),axis=1)

In [385]:
xticks = np.linspace(0.0,1.0,3)
xlims = [-0.1,1.1]
yticklambda = np.linspace(-5,5,3)
ylimlambda = [-5.1,5.1]
ytickpc = np.linspace(0.5,1.0,2) # homogeneous
# ytickpc = np.linspace(0.70,1.0,2) # RII
ylimpc = [0.49,1.01] # homogeneous
# ylimpc = [0.69,1.01] # RII
fig,ax=plt.subplots(1,3,figsize=(12,3))
# FIGURE 1
for i in range(ngavg):
    for j in range(nrank):
        # RECONSTRUCT EIGENVALUES
        # m1
        ax[0].scatter(gaverageseries[i]*np.ones(nbatch),lambda_reconstruct[i,0,j,:],s=2,color='orange',alpha=0.5)
        ax[0].scatter(gaverageseries[i]*np.ones(nbatch),lambda_reconstruct[i,1,j,:],s=2,color='cyan',alpha=0.5)
        ax[0].scatter(gaverageseries[i]*np.ones(nbatch),lambda_reconstruct[i,2,j,:],s=2,color='gray',alpha=0.5)

        
        # OVERLAP
        # m1
        # ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),overlapeig[i,0,j,:],s=5,color='red',alpha=0.5)
        # ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),overlapeig[i,1,j,:],s=5,color='blue',alpha=0.5)
        ax[1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,0,j,:]),s=2,color='orange',alpha=0.5)
        ax[1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,1,j,:]),s=2,color='cyan',alpha=0.5)
        ax[1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,2,j,:]),s=2,color='gray',alpha=0.5)


        # n1
        ax[2].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,0,j+nrank,:]),s=2,color='orange',alpha=0.5)
        ax[2].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,1,j+nrank,:]),s=2,color='cyan',alpha=0.5)
        ax[2].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,2,j+nrank,:]),s=2,color='gray',alpha=0.5)

for j in range(nrank):
    lambdaE = np.mean(np.squeeze(lambda_reconstruct[:,0,j,:]),axis=1)
    lambdaI = np.mean(np.squeeze(lambda_reconstruct[:,1,j,:]),axis=1)
    lambdaT = np.mean(np.squeeze(lambda_reconstruct[:,2,j,:]),axis=1)
    ax[0].plot(gaverageseries,lambdaE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[0].plot(gaverageseries,lambdaI,color='blue',linewidth=1.5,alpha=1.0,label='E population')
    ax[0].plot(gaverageseries,lambdaT,color='black',linewidth=1.5,alpha=1.0,label='E population')
    ax[0].set_xlim(xlims)
    ax[0].set_ylim(ylimlambda)
    ax[0].set_xticks(xticks)
    ax[0].set_yticks(yticklambda)

    mE = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,0,j,:])),axis=1)
    mI = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,1,j,:])),axis=1)
    mT = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,2,j,:])),axis=1)
    ax[1].plot(gaverageseries,mE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[1].plot(gaverageseries,mI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[1].plot(gaverageseries,mT,color='black',linewidth=1.5,alpha=1.0,label='Total')
    ax[1].set_xlim(xlims)
    ax[1].set_ylim(ylimpc)
    ax[1].set_xticks(xticks)
    ax[1].set_yticks(ytickpc)

    nE = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,0,j+nrank,:])),axis=1)
    nI = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,1,j+nrank,:])),axis=1)
    nT = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,2,j+nrank,:])),axis=1)
    ax[2].plot(gaverageseries,nE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[2].plot(gaverageseries,nI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[2].plot(gaverageseries,nT,color='black',linewidth=1.5,alpha=1.0,label='Total')
    ax[2].set_xlim(xlims)
    ax[2].set_ylim(ylimpc)
    ax[2].set_xticks(xticks)
    ax[2].set_yticks(ytickpc)
ax[2].legend()

In [386]:
ytickavg = np.linspace(0.020,0.032,3)
ylimavg = [0.0199,0.0321]
fig,ax=plt.subplots(1,2,figsize=(12,3))
# FIGURE same as overlap
for i in range(ngavg):
    for j in range(nrank):
        # mean m1
        ax[0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,j,:])/np.sqrt(Norm2[i,0,j,:]),s=5,color='orange',alpha=0.25)
        ax[0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,j,:])/np.sqrt(Norm2[i,1,j,:]),s=5,color='cyan',alpha=0.25)
        # mean n1
        ax[1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,nrank+j,:])/np.sqrt(Norm2[i,0,j+nrank,:]),s=5,color='orange',alpha=0.25)
        ax[1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,nrank+j,:])/np.sqrt(Norm2[i,1,j+nrank,:]),s=5,color='cyan',alpha=0.25)

for j in range(nrank):
    mmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,0,j,:]))/np.squeeze(np.sqrt(Norm2[:,0,j,:])),axis=1)
    mmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,1,j,:]))/np.squeeze(np.sqrt(Norm2[:,1,j,:])),axis=1)
    ax[0].plot(gaverageseries,mmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[0].plot(gaverageseries,mmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[0].set_xlim(xlims)
    ax[0].set_ylim(ylimavg)
    ax[0].set_xticks(xticks)
    ax[0].set_yticks(ytickavg)

    nmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,0,j+nrank,:]))/np.squeeze(np.sqrt(Norm2[:,0,j+nrank,:])),axis=1)
    nmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,1,j+nrank,:]))/np.squeeze(np.sqrt(Norm2[:,1,j+nrank,:])),axis=1)
    ax[1].plot(gaverageseries,nmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[1].plot(gaverageseries,nmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[1].set_xlim(xlims)
    ax[1].set_ylim(ylimavg)
    ax[1].set_xticks(xticks)
    ax[1].set_yticks(ytickavg)
ax[1].legend()
fig,ax=plt.subplots(1,2,figsize=(12,3))
# FIGURE absolute value
for i in range(ngavg):
    for j in range(nrank):
        # mean m1
        ax[0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,j,:]),s=5,color='red',alpha=0.5)
        ax[0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,j,:]),s=5,color='blue',alpha=0.5)
        # mean n1
        ax[1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,nrank+j,:]),s=5,color='red',alpha=0.5)
        ax[1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,nrank+j,:]),s=5,color='blue',alpha=0.5)

In [414]:
# statistics
mAm2,nAm2=np.zeros(2),np.zeros(2)
for i in range(nrank):
    mAm2[:]=meigvecAm[1::NE,i]
    nAm2[:]=neigvecAm[1::NE,i]
print('rank-1',mAm2,nAm2)
meanloadingeig_trialsavg=np.zeros((ngavg,2,2*nrank))
# m1--
meanloadingeig_trialsavg[:,:,0] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,0,:])),axis=2)#np.squeeze(meanloadingeigclouds[:,:,:,12])#
meanloadingeig_trialsavg[:,:,1] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,1,:])),axis=2)
meanloadingeig_trialsavg[:,1,1]*=-1
print('mean:',np.shape(meanloadingeig_trialsavg))
Oeigseries=np.zeros((ngavg,2,nrank*2))
for i in range(ngavg):
    m_mean = np.squeeze(meanloadingeig_trialsavg[i,:,:nrank])
    n_mean = np.squeeze(meanloadingeig_trialsavg[i,:,nrank:])
    print('num',m_mean,n_mean)
    Oeigm,Oeign = m_mean[:]/mAm2[:],n_mean[:]/nAm2[:]
    Oeigseries[i,:,:nrank],Oeigseries[i,:,nrank:]=np.reshape(Oeigm.copy(),(2,nrank)),np.reshape(Oeign.copy(),(2,nrank))

# three copy
idx = 6
grand = gaverageseries[idx]*np.ones(4)*np.sqrt(2.0/3.0)
grand[0]=0
xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
eigvgm,eigvecgm=la.eig(gmat)
# print(eigvgm)
print(Oeigseries[idx,:,:nrank])
print(Oeigseries[idx,:,nrank:])
attt=(1-gaverageseries[idx]**2/eigvAm[:nrank]**2)#eigvgm**2/eigvAm[:nrank]**2)
print(np.sqrt(attt))
fig,ax=plt.subplots(figsize=(5,5))
# FIGURE absolute value
ax.plot(gaverageseries,Oeigseries[:,0,0],color='red',linewidth=1.5,label=r'$\alpha_{m}^E$')
ax.plot(gaverageseries,Oeigseries[:,1,0],color='blue',linewidth=1.5,label=r'$\alpha_{m}^I$')
ax.plot(gaverageseries,Oeigseries[:,0,1],color='orange',linewidth=1.5,label=r'$\alpha_{n}^E$')
ax.plot(gaverageseries,Oeigseries[:,1,1],color='cyan',linewidth=1.5,label=r'$\alpha_{n}^I$')
ax.plot(gaverageseries,np.sqrt(1-gaverageseries**2/eigvAm[0]**2),color='black',linestyle='--',linewidth=1.5,label=r'$\alpha_m^{theo}$')
ax.plot(gaverageseries,1/np.sqrt(1-gaverageseries**2/eigvAm[0]**2),color='gray',linestyle='--',linewidth=1.5,label=r'$\alpha_n^{theo}$')
yalims=[0.89,1.11]
yaticks=np.linspace(0.9,1.1,3)
ax.set_xlim(xlims)
ax.set_xticks(xticks)
ax.set_ylim(yalims)
ax.set_yticks(yaticks)
ax.legend()


[[0.98856498]
 [0.99179441]]
[[1.00312094]
 [1.00737401]]
[0.9901515+0.j]


# RANK-2

In [445]:

''' RANK-2 '''
''' heterogeneous case with single zero g_{EE} '''
# strfile = '2021-01-20-10_38_16_heterogeneousgIIR2_cloudsdata'
# strfile = '2021-01-20-11_12_48_homogeneousR2_cloudsdata'
# strfile ='2021-01-20-11_41_35_heterogeneousNotgEER2_cloudsdata'
strfile='2021-01-23-16_18_40_heterogeneousNotgEER2_cloudsdata'
# strfile='2021-01-23-17_02_52_homogeneousR2_cloudsdata'
# strfile = '2021-01-24-20_42_28_heterogeneousrIIR2_cloudsdata'
data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
meigseries,neigseries=data['meigseries'],data['neigseries']
gaverageseries=data['gaverageseries']
# # loading calculated mean and std (2*2) for rank1, 0,0 m,E 0,1 m,I
# meanloadingseries,stdloadingseries=data['meanloadingseries'],data['stdloadingseries']
# ((niter,ntrials,NE*2))
ngavg,nbatch,_,_=np.shape(meigseries)
nrank=2
Jparameters=data['Jparameters']
NE = int(Jparameters[4])

P = np.zeros((2,2))
P = np.array([[Jparameters[0],-Jparameters[1]],[Jparameters[2],-Jparameters[3]]])
Am=np.zeros((NE*2,NE*2))
Am[:NE,:NE],Am[:NE,NE:]=P[0,0]/NE,P[0,1]/NE
Am[NE:,:NE],Am[NE:,NE:]=P[1,0]/NE,P[1,1]/NE
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('\bar{m}&\bar{n}')
print(meigvecAm[1::NE,0],meigvecAm[1::NE,1])
print(neigvecAm[1::NE,0],neigvecAm[1::NE,1])
print('eigenvalues:',eigvAm[:nrank])
print(np.sum(neigvecAm[:,0]*meigvecAm[:,0]))
print(np.sum(neigvecAm[:,1]*meigvecAm[:,1]))

ar{m}ar{n}
[-0.01661519+0.j -0.02690605+0.j] [0.02650294+0.j 0.01725092+0.j]
[-0.19787336+0.j  0.30399677-0.j] [-0.08254572+0.j  0.05097416-0.j]
eigenvalues: [-4.89164729+0.j -1.30835271+0.j]
(-4.8916472867170295+0j)
(-1.3083527132830093+0j)


In [293]:
idxgavg=10
# generate figure
xticks = np.linspace(-4.0,1.0,5)
xlims = [-4.1,1.1]
yticks = np.linspace(-0.8,0.8,3)
ylims = [-0.81,0.81]
theta = np.linspace(0, 2 * np.pi, 200)
# fig,ax=plt.subplots(1,3,figsize=(12,4))
# ''' PLOT a sample scatter plot A '''
# ## three cases
# case one homogeneous
strfile = '2021-01-20-11_12_48_homogeneousR2_cloudsdata'
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
nrank=2
Jparameters=data['Jparameters']
JEE,JIE,JEI,JII=Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]
Nparams=np.array([NE,NE])
# generate mean 
Am,Jsv,ua,svam,va=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# OVERALL
eigvalAm,_=la.eig(Am)
# generate X
# totally average
gaverage=gaverageseries[idxgavg]
grand = gaverage*np.ones(4)/np.sqrt(2.0)
print('pre avg:',gaverage)
## homogeneous X matrix
X,Gamp,eigvgmT,eigvecgm,gaverage=generate_localstatsmat_eig(Nparams,grand)#,ugm,svgm,vgm
print('post avg:',gaverage)
xr = gaverage*np.cos(theta)
yr = gaverage*np.sin(theta)
# mix together
Xtotal=X.copy()+Am.copy()
eigvxt,eigvecxt=la.eig(Xtotal)
# eigenvalues
eigvalseries=data['eigvalseries']

fig,ax=plt.subplots(3,1,figsize=(8,8))
real_eigsbulk,imag_eigsbulk=np.real(eigvxt[2:]),np.imag(eigvxt[2:])
ax[0].scatter(real_eigsbulk,imag_eigsbulk,marker='^',s=5,color='blue',alpha=0.25)
ax[0].scatter(np.real(eigvxt[:2]),np.imag(eigvxt[:2]),marker='^',s=5,color='blue',alpha=0.5)
for i in range(nrank):
    ax[0].scatter(np.real(eigvalseries[idxgavg,:,i]),np.imag(eigvalseries[idxgavg,:,i]),marker='^',s=5,color='blue',alpha=0.5)
xa = gaverage*np.cos(theta)
ya = gaverage*np.sin(theta)
ax[0].plot(xa, ya, color="darkred", linewidth=1.5,label=r'$r_g$')
ax[0].plot(xr, yr, color="grey", linewidth=1.5,linestyle='--',label=r'fixed $\bar{g}$')
ax[0].scatter(np.real(eigvalAm[:2]),np.imag(eigvalAm[:2]),marker='^',s=15,color='red',alpha=1.0)
ax[0].set_xlim(xlims)
ax[0].set_ylim(ylims)
ax[0].set_xticks(xticks)
ax[0].set_yticks(yticks)
ax[0].set_aspect('equal')
ax[0].legend()
ax[0].grid()

# case two heterogeneous RII
strfile = '2021-01-20-10_38_16_heterogeneousgIIR2_cloudsdata'
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
# generate X
# max r_g
grand = np.zeros(4)
grand[3]=gaverage*np.sqrt(2) # I dominant
print('pre avg:',gaverage)
## homogeneous X matrix
X,Gamp,eigvgmT,eigvecgm,gaverage=generate_localstatsmat_eig(Nparams,grand)#,ugm,svgm,vgm
print('post avg:',gaverage)

# mix together
Xtotal=X.copy()+Am.copy()
eigvxt,eigvecxt=la.eig(Xtotal)
# eigenvalues
eigvalseries=data['eigvalseries']

real_eigsbulk,imag_eigsbulk=np.real(eigvxt[2:]),np.imag(eigvxt[2:])
ax[1].scatter(real_eigsbulk,imag_eigsbulk,marker='^',s=5,color='blue',alpha=0.25)
ax[1].scatter(np.real(eigvxt[:2]),np.imag(eigvxt[:2]),marker='^',s=5,color='blue',alpha=0.5)
for i in range(nrank):
    ax[1].scatter(np.real(eigvalseries[idxgavg,:,i]),np.imag(eigvalseries[idxgavg,:,i]),marker='^',s=5,color='blue',alpha=0.5)
xa = np.sqrt(2)*gaverage*np.cos(theta)
ya = np.sqrt(2)*gaverage*np.sin(theta)
ax[1].plot(xa, ya, color="darkred", linewidth=1.5,label=r'$r_g$')
ax[1].plot(xr, yr, color="grey", linewidth=1.5,linestyle='--',label=r'fixed $\bar{g}$')
ax[1].scatter(np.real(eigvalAm[:2]),np.imag(eigvalAm[:2]),marker='^',s=15,color='red',alpha=1.0)
ax[1].set_xlim(xlims)
ax[1].set_ylim(ylims)
ax[1].set_xticks(xticks)
ax[1].set_yticks(yticks)
ax[1].set_aspect('equal')
ax[0].set_aspect('equal')
ax[1].legend()
ax[1].grid()

# case two heterogeneous not gEE
strfile = '2021-01-20-11_41_35_heterogeneousNotgEER2_cloudsdata'
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
# generate X
# three copy
grand = gaverage*np.ones(4)*np.sqrt(2.0/3.0)
grand[0]=0
print('pre avg:',gaverage)
## homogeneous X matrix
X,Gamp,eigvgmT,eigvecgm,gaverage=generate_localstatsmat_eig(Nparams,grand)#,ugm,svgm,vgm
print('post avg:',gaverage)

# mix together
Xtotal=X.copy()+Am.copy()
eigvxt,eigvecxt=la.eig(Xtotal)
# eigenvalues
eigvalseries=data['eigvalseries']

real_eigsbulk,imag_eigsbulk=np.real(eigvxt[2:]),np.imag(eigvxt[2:])
ax[2].scatter(real_eigsbulk,imag_eigsbulk,marker='^',s=5,color='blue',alpha=0.25)
ax[2].scatter(np.real(eigvxt[:2]),np.imag(eigvxt[:2]),marker='^',s=5,color='blue',alpha=0.5)
for i in range(nrank):
    ax[2].scatter(np.real(eigvalseries[idxgavg,:,i]),np.imag(eigvalseries[idxgavg,:,i]),marker='^',s=5,color='blue',alpha=0.5)
arg = np.sqrt((1+np.sqrt(5))/3.0)
xa = arg*gaverage*np.cos(theta)
ya = arg*gaverage*np.sin(theta)
ax[2].plot(xa, ya, color="darkred", linewidth=1.5,label=r'$r_g$')
ax[2].plot(xr, yr, color="grey", linewidth=1.5,linestyle='--',label=r'fixed $\bar{g}$')
ax[2].scatter(np.real(eigvalAm[:2]),np.imag(eigvalAm[:2]),marker='^',s=15,color='red',alpha=1.0)
ax[2].set_xlim(xlims)
ax[2].set_ylim(ylims)
ax[2].set_xticks(xticks)
ax[2].set_yticks(yticks)
ax[2].set_aspect('equal')
ax[0].set_aspect('equal')
ax[2].legend()
ax[2].grid()




pre avg: 0.5499999999999999
post avg: 0.5499999999999998
pre avg: 0.5499999999999998
post avg: 0.5499999999999998
pre avg: 0.5499999999999998
post avg: 0.5499999999999998


In [446]:
''' For rank 2 '''
# calculate \theta overlap
loadingeigvec=np.zeros((ngavg,2,nrank*2,NE,nbatch)) # ng,EI,nrank*2 mn,Npopulation,ntrials
loadingeigvecAm=np.zeros((2,nrank*2,NE))
meanloadingeigclouds,stdloadingeigclouds = np.zeros((ngavg,2,nrank*2,nbatch)),np.zeros((ngavg,2,nrank*2,nbatch))
Norm2=np.zeros((ngavg,3,nrank*2,nbatch))
Norm2Am=np.zeros((3,nrank*2,nbatch))
overlapeig,overlapeig_renorm=np.zeros((ngavg,3,nrank*2,nbatch)),np.zeros((ngavg,3,nrank*2,nbatch))
# lambda reconstruction
lambda_reconstruct=np.zeros((ngavg,3,nrank,nbatch)) # E pop, I pop, overall
for j in range(nrank):
    # eig
    loadingeigvecAm[0,j,:]=np.squeeze(meigvecAm[:NE,j]).T # m_j^E
    loadingeigvecAm[0,j+nrank,:]=np.squeeze(neigvecAm[:NE,j]).T # n_j^E
    loadingeigvecAm[1,j,:]=np.squeeze(meigvecAm[NE:,j]).T # m_j^I
    loadingeigvecAm[1,j+nrank,:]=np.squeeze(neigvecAm[NE:,j]).T # n_j^I
    Norm2Am[0,j]=np.sum(np.squeeze(loadingeigvecAm[0,j,:])**2,axis=0) # excitatory
    Norm2Am[1,j]=np.sum(np.squeeze(loadingeigvecAm[1,j,:])**2,axis=0) # inhibitory
    Norm2Am[2,j]=Norm2Am[1,j]+Norm2Am[0,j] # overall
    Norm2Am[0,j+nrank]=np.sum(np.squeeze(loadingeigvecAm[0,j+nrank,:])**2,axis=0) # excitatory
    Norm2Am[1,j+nrank]=np.sum(np.squeeze(loadingeigvecAm[1,j+nrank,:])**2,axis=0) # inhibitory
    Norm2Am[2,j+nrank]=Norm2Am[1,j+nrank]+Norm2Am[0,j+nrank] # overall
for j in range(nrank):
    for i in range(ngavg):
        # eig
        loadingeigvec[i,0,j,:,:]=np.squeeze(meigseries[i,:,:NE,j]).T # m_j^E
        loadingeigvec[i,0,j+nrank,:,:]=np.squeeze(neigseries[i,:,:NE,j]).T # n_j^E
        loadingeigvec[i,1,j,:,:]=np.squeeze(meigseries[i,:,NE:,j]).T # m_j^I
        loadingeigvec[i,1,j+nrank,:,:]=np.squeeze(neigseries[i,:,NE:,j]).T # n_j^I

        # renormalization
        # m, right eigenvectors
        Norm2[i,0,j,:]=np.sum(np.squeeze(loadingeigvec[i,0,j,:,:])**2,axis=0) # excitatory
        Norm2[i,1,j,:]=np.sum(np.squeeze(loadingeigvec[i,1,j,:,:])**2,axis=0) # inhibitory
        Norm2[i,2,j,:]=Norm2[i,1,j,:]+Norm2[i,0,j,:] # overall
        # n, left eigenvectors
        Norm2[i,0,j+nrank,:]=np.sum(np.squeeze(loadingeigvec[i,0,j+nrank,:,:])**2,axis=0) # excitatory n
        Norm2[i,1,j+nrank,:]=np.sum(np.squeeze(loadingeigvec[i,1,j+nrank,:,:])**2,axis=0) # inhibitory n
        Norm2[i,2,j+nrank,:]=Norm2[i,1,j+nrank,:]+Norm2[i,0,j+nrank,:] # overall normalization
        # overlap
        # m
        overlapeig[i,0,j,:]=np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:])) # excitatory overlap -- renormalized
        overlapeig[i,1,j,:]=np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:])) # inhibitory overlap -- renromalized
        overlapeig[i,2,j,:]=np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:]))+np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:])) # inhibitory overlap -- renormalized
        # n
        overlapeig[i,0,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))/(eigvAm[j])**2 # excitatory overlap -- renormalized
        overlapeig[i,1,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:]))/(eigvAm[j])**2# inhibitory overlap -- renormalize+
        overlapeig[i,2,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))/(eigvAm[j])**2+np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:]))/(eigvAm[j])**2# inhibitory overlap -- renormalized

        # overlap renormalizing
        # m
        overlapeig_renorm[i,0,j,:]=np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:]))/np.sqrt(Norm2[i,0,j,:])/np.sqrt(Norm2Am[0,j]) # excitatory overlap -- renormalized
        overlapeig_renorm[i,1,j,:]=np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:]))/np.sqrt(Norm2[i,1,j,:])/np.sqrt(Norm2Am[1,j]) # inhibitory overlap -- renromalized
        overlapeig_renorm[i,2,j,:]=(np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:]))+np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:])))/np.sqrt(Norm2[i,2,j,:])/np.sqrt(Norm2Am[2,j]) # overlap -- renromalized
        # n
        overlapeig_renorm[i,0,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))/np.sqrt(Norm2[i,0,j+nrank,:])/np.sqrt(Norm2Am[0,j+nrank]) # excitatory overlap -- renormalized
        overlapeig_renorm[i,1,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:]))/np.sqrt(Norm2[i,1,j+nrank,:])/np.sqrt(Norm2Am[1,j+nrank]) # inhibitory overlap -- renormalized
        overlapeig_renorm[i,2,j+nrank,:]=(np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))+np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:])))/np.sqrt(Norm2[i,2,j+nrank,:])/np.sqrt(Norm2Am[2,j+nrank])
        # statistics
        meanloadingeigclouds[i,0,j,:]=(np.mean(np.squeeze(loadingeigvec[i,0,j,:,:]),axis=0)) # m excitatory trial averaging
        meanloadingeigclouds[i,0,j+nrank,:]=(np.mean(np.squeeze(loadingeigvec[i,0,j+nrank,:,:]),axis=0)) # n excitatory trial average
        meanloadingeigclouds[i,1,j,:]=(np.mean(np.squeeze(loadingeigvec[i,1,j,:,:]),axis=0)) # m inhibitory trial average
        meanloadingeigclouds[i,1,j+nrank,:]=(np.mean(np.squeeze(loadingeigvec[i,1,j+nrank,:,:]),axis=0)) # n inhibitory trial average

        stdloadingeigclouds[i,0,j,:]=np.std(np.squeeze(loadingeigvec[i,0,j,:,:]),axis=0)
        stdloadingeigclouds[i,0,j+nrank,:]=np.std(np.squeeze(loadingeigvec[i,0,j+nrank,:,:]),axis=0)
        stdloadingeigclouds[i,1,j,:]=np.std(np.squeeze(loadingeigvec[i,1,j,:,:]),axis=0)
        stdloadingeigclouds[i,1,j+nrank,:]=np.std(np.squeeze(loadingeigvec[i,1,j+nrank,:,:]),axis=0)

        # reproduce lambda overall
        lambda_reconstruct[i,0,j,:]=np.sum(np.squeeze(meigseries[i,:,:NE,j])*np.squeeze(neigseries[i,:,:NE,j]),axis=1)
        lambda_reconstruct[i,1,j,:]=np.sum(np.squeeze(meigseries[i,:,NE:,j])*np.squeeze(neigseries[i,:,NE:,j]),axis=1)
        lambda_reconstruct[i,2,j,:]=np.sum(np.squeeze(meigseries[i,:,:,j])*np.squeeze(neigseries[i,:,:,j]),axis=1)

## Plottings for eigenvectors 
* overlap of right and left eigenvectors -- $\lambda_1,\lambda_2$
* overlap of the new and original eigenvectors -- renormalize?  
* connectivity patterns in loading space -- mean and variance of gaussian pattern

In [447]:
xticks = np.linspace(0.0,1.0,3)
xlims = [-0.1,1.1]
yticklambda = np.linspace(-15,15,3)
ylimlambda = [-15.1,15.1]
# ytickpc = np.linspace(0.75,1.0,2) # homogeneous
# ytickpc = np.linspace(0.70,1.0,2) # RII
ytickpc = np.linspace(0.50,1.0,2) # new RII
# ylimpc = [0.74,1.01] # homogeneous
# ylimpc = [0.69,1.01] # RII
ylimpc = [0.49,1.01] # new RII
fig,ax=plt.subplots(2,3,figsize=(12,4))
# FIGURE 1
for i in range(ngavg):
    for j in range(nrank):
        # RECONSTRUCT EIGENVALUES
        # m1
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),lambda_reconstruct[i,0,j,:],s=2,color='orange',alpha=0.5)
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),lambda_reconstruct[i,1,j,:],s=2,color='cyan',alpha=0.5)
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),lambda_reconstruct[i,2,j,:],s=2,color='gray',alpha=0.5)

        
        # OVERLAP
        # m1
        # ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),overlapeig[i,0,j,:],s=5,color='red',alpha=0.5)
        # ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),overlapeig[i,1,j,:],s=5,color='blue',alpha=0.5)
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,0,j,:]),s=2,color='orange',alpha=0.5)
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,1,j,:]),s=2,color='cyan',alpha=0.5)
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,2,j,:]),s=2,color='gray',alpha=0.5)


        # n1
        ax[j][2].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,0,j+nrank,:]),s=2,color='orange',alpha=0.5)
        ax[j][2].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,1,j+nrank,:]),s=2,color='cyan',alpha=0.5)
        ax[j][2].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(overlapeig_renorm[i,2,j+nrank,:]),s=2,color='gray',alpha=0.5)

for j in range(nrank):
    lambdaE = np.mean(np.squeeze(lambda_reconstruct[:,0,j,:]),axis=1)
    lambdaI = np.mean(np.squeeze(lambda_reconstruct[:,1,j,:]),axis=1)
    lambdaT = np.mean(np.squeeze(lambda_reconstruct[:,2,j,:]),axis=1)
    ax[j][0].plot(gaverageseries,lambdaE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[j][0].plot(gaverageseries,lambdaI,color='blue',linewidth=1.5,alpha=1.0,label='E population')
    ax[j][0].plot(gaverageseries,lambdaT,color='black',linewidth=1.5,alpha=1.0,label='E population')
    ax[j][0].set_xlim(xlims)
    ax[j][0].set_ylim(ylimlambda)
    ax[j][0].set_xticks(xticks)
    ax[j][0].set_yticks(yticklambda)

    mE = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,0,j,:])),axis=1)
    mI = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,1,j,:])),axis=1)
    mT = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,2,j,:])),axis=1)
    ax[j][1].plot(gaverageseries,mE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[j][1].plot(gaverageseries,mI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[j][1].plot(gaverageseries,mT,color='black',linewidth=1.5,alpha=1.0,label='Total')
    ax[j][1].set_xlim(xlims)
    ax[j][1].set_ylim(ylimpc)
    ax[j][1].set_xticks(xticks)
    ax[j][1].set_yticks(ytickpc)

    nE = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,0,j+nrank,:])),axis=1)
    nI = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,1,j+nrank,:])),axis=1)
    nT = np.mean(np.squeeze(np.abs(overlapeig_renorm[:,2,j+nrank,:])),axis=1)
    ax[j][2].plot(gaverageseries,nE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[j][2].plot(gaverageseries,nI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[j][2].plot(gaverageseries,nT,color='black',linewidth=1.5,alpha=1.0,label='Total')
    ax[j][2].set_xlim(xlims)
    ax[j][2].set_ylim(ylimpc)
    ax[j][2].set_xticks(xticks)
    ax[j][2].set_yticks(ytickpc)
ax[0][2].legend()

In [17]:
print(np.mean(lambda_reconstruct[:,2,0,:]))

[-3.19848481 -3.19560433 -3.23030985 -3.21675155 -3.21975556 -3.17560632
 -3.2131477  -3.0465627  -3.17194874 -3.23273991 -3.14884097 -3.18938156
 -3.20051234 -3.21066659 -3.19976068 -3.18586634 -3.20226587 -3.19118601
 -3.2157759  -3.16754911]


In [448]:
# ytickavg = np.linspace(0.022,0.032,3)
# ylimavg = [0.0219,0.0321]
# new R2
ytickavg = np.linspace(0.002,0.032,3)
ylimavg = [0.0019,0.0321]
fig,ax=plt.subplots(2,2,figsize=(12,4))
# FIGURE same as overlap
for i in range(ngavg):
    for j in range(nrank):
        # mean m1
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,j,:])/np.sqrt(Norm2[i,0,j,:]),s=5,color='orange',alpha=0.25)
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,j,:])/np.sqrt(Norm2[i,1,j,:]),s=5,color='cyan',alpha=0.25)
        # mean n1
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,nrank+j,:])/np.sqrt(Norm2[i,0,j+nrank,:]),s=5,color='orange',alpha=0.25)
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,nrank+j,:])/np.sqrt(Norm2[i,1,j+nrank,:]),s=5,color='cyan',alpha=0.25)

for j in range(nrank):
    mmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,0,j,:]))/np.squeeze(np.sqrt(Norm2[:,0,j,:])),axis=1)
    mmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,1,j,:]))/np.squeeze(np.sqrt(Norm2[:,1,j,:])),axis=1)
    ax[j][0].plot(gaverageseries,mmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[j][0].plot(gaverageseries,mmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[j][0].set_xlim(xlims)
    ax[j][0].set_ylim(ylimavg)
    ax[j][0].set_xticks(xticks)
    ax[j][0].set_yticks(ytickavg)

    nmeanE = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,0,j+nrank,:]))/np.squeeze(np.sqrt(Norm2[:,0,j+nrank,:])),axis=1)
    nmeanI = np.mean(np.squeeze(np.abs(meanloadingeigclouds[:,1,j+nrank,:]))/np.squeeze(np.sqrt(Norm2[:,1,j+nrank,:])),axis=1)
    ax[j][1].plot(gaverageseries,nmeanE,color='red',linewidth=1.5,alpha=1.0,label='E population')
    ax[j][1].plot(gaverageseries,nmeanI,color='blue',linewidth=1.5,alpha=1.0,label='I population')
    ax[j][1].set_xlim(xlims)
    ax[j][1].set_ylim(ylimavg)
    ax[j][1].set_xticks(xticks)
    ax[j][1].set_yticks(ytickavg)
ax[0][1].legend()
fig,ax=plt.subplots(2,2,figsize=(12,4))
# FIGURE absolute value
for i in range(ngavg):
    for j in range(nrank):
        # mean m1
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,j,:]),s=5,color='red',alpha=0.5)
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,j,:]),s=5,color='blue',alpha=0.5)
        # mean n1
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,nrank+j,:]),s=5,color='red',alpha=0.5)
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,nrank+j,:]),s=5,color='blue',alpha=0.5)

## quantitative measurements

In [449]:
# statistics
mAm2,nAm2=np.zeros((2,2)),np.zeros((2,2))
for i in range(nrank):
    mAm2[:,i]=meigvecAm[1::NE,i]
    nAm2[:,i]=neigvecAm[1::NE,i]
invmbar,invnbar=la.inv(mAm2),la.inv(nAm2)
meanloadingeig_trialsavg=np.zeros((ngavg,2,nrank*2))
# ''' a=b change '''
# # m1--
# meanloadingeig_trialsavg[:,:,0] = -np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,0,:])),axis=2)#np.squeeze(meanloadingeigclouds[:,:,:,12])#
# meanloadingeig_trialsavg[:,:,1] = -np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,1,:])),axis=2)
# meanloadingeig_trialsavg[:,:,2] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,2,:])),axis=2)
# meanloadingeig_trialsavg[:,0,2]*=-1
# meanloadingeig_trialsavg[:,:,3] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,3,:])),axis=2)
# meanloadingeig_trialsavg[:,1,3]*=-1
''' a!=b '''
# m1--
meanloadingeig_trialsavg[:,:,0] = -np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,0,:])),axis=2)#np.squeeze(meanloadingeigclouds[:,:,:,12])#
meanloadingeig_trialsavg[:,:,1] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,1,:])),axis=2)
meanloadingeig_trialsavg[:,:,2] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,2,:])),axis=2)
meanloadingeig_trialsavg[:,0,2]*=-1
meanloadingeig_trialsavg[:,:,3] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,3,:])),axis=2)
meanloadingeig_trialsavg[:,0,3]*=-1
Oeigseries=np.zeros((ngavg,2,nrank*2))
for i in range(ngavg):
    m_mean = np.squeeze(meanloadingeig_trialsavg[i,:,:nrank])
    n_mean = np.squeeze(meanloadingeig_trialsavg[i,:,nrank:])
    Oeigm,Oeign = invmbar@m_mean,invnbar@n_mean
    Oeigseries[i,:,:nrank],Oeigseries[i,:,nrank:]=Oeigm.copy(),Oeign.copy()

# three copy
idx = 9
grand = gaverageseries[idx]*np.ones(4)*np.sqrt(2.0/3.0)
grand[0]=0
xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
eigvgm,eigvecgm=la.eig(gmat)
# print(eigvgm)
print(Oeigseries[idx,:,:nrank])
print(Oeigseries[idx,:,nrank:])
attt=(1-gaverageseries[idx]**2/eigvAm[:nrank]**2)#eigvgm**2/eigvAm[:nrank]**2)
print(np.sqrt(attt))
## plot
fig,ax=plt.subplots(1,2,figsize=(12,5))
''' a=b ''' 
# yalims=[0.89,1.11]
# yaticks=np.linspace(0.9,1.1,3)
''' a!=b ''' 
yalims=[0.49,1.51]
yaticks=np.linspace(0.5,1.5,3)
for i in range(nrank):
    # FIGURE absolute value
    ax[i].plot(gaverageseries,Oeigseries[:,i,i],color='red',linewidth=1.5,label=r'$\alpha_{m}^E$')
    ax[i].plot(gaverageseries,Oeigseries[:,i,i+nrank],color='blue',linewidth=1.5,label=r'$\alpha_{n}^I$')
    ax[i].plot(gaverageseries,np.sqrt(1-gaverageseries**2/eigvAm[i]**2),color='black',linestyle='--',linewidth=1.5,label=r'$\alpha_m^{theo}$')
    ax[i].plot(gaverageseries,1/np.sqrt(1-gaverageseries**2/eigvAm[i]**2),color='gray',linestyle='--',linewidth=1.5,label=r'$\alpha_n^{theo}$')
    ax[i].set_xlim(xlims)
    ax[i].set_xticks(xticks)
    ax[i].set_ylim(yalims)
    ax[i].set_yticks(yaticks)
ax[1].legend()


[[9.96470364e-01 9.54258472e-04]
 [2.71011811e-03 9.32467920e-01]]
[[ 1.00471985e+00 -7.97712945e-04]
 [-3.28235789e-03  1.06799455e+00]]
[0.99476232+0.j 0.92409619+0.j]


In [433]:
Oeigseries[:,1,]

array([-1.00025096, -1.00134822, -1.00213784, -1.00730093, -1.00985742,
       -1.01564095, -1.02462289, -1.03230674, -1.05014872, -1.0595593 ,
       -1.0829692 , -1.10449351, -1.14307025, -1.20408384, -1.26507768,
       -1.40173683, -1.05138942, -0.57342441, -0.33823233, -0.1994065 ])

In [428]:
for i in range(20):
    mEnum, mInum=Oeigseries[i,:,:nrank],Oeigseries[i,:,nrank:]
    print(mEnum,mInum.T)

[[ 9.99989135e-01 -7.33584433e-05]
 [ 7.36914413e-05 -9.99620136e-01]] [[ 1.00009843e+00 -2.80943594e-04]
 [ 1.88603752e-05 -1.00025096e+00]]
[[ 9.99840230e-01 -1.60651147e-04]
 [ 1.59165680e-04 -9.98382836e-01]] [[ 1.00035020e+00 -6.04223587e-04]
 [ 4.02797860e-05 -1.00134822e+00]]
[[ 9.99951197e-01 -7.48988959e-04]
 [ 7.02870499e-04 -9.96649661e-01]] [[ 1.00085955e+00 -2.68126980e-03]
 [ 1.93546437e-04 -1.00213784e+00]]
[[ 9.98801440e-01  6.64961837e-05]
 [ 1.76782252e-05 -9.92680871e-01]] [[ 1.00126273e+00 -5.42310235e-05]
 [-2.30841190e-05 -1.00730093e+00]]
[[ 9.98678931e-01 -6.88060676e-04]
 [ 6.08736947e-04 -9.89251717e-01]] [[ 1.00201526e+00 -2.31459766e-03]
 [ 1.75644484e-04 -1.00985742e+00]]
[[ 9.97888083e-01 -7.63328523e-04]
 [ 6.51530169e-04 -9.83838737e-01]] [[ 1.00291409e+00 -2.43298659e-03]
 [ 1.77021307e-04 -1.01564095e+00]]
[[ 9.96370010e-01 -4.01363770e-04]
 [ 1.47867804e-04 -9.76027446e-01]] [[ 1.00376127e+00 -5.20511223e-04]
 [ 9.14584683e-05 -1.02462289e+00]]
[[ 9.9

In [425]:
fig,ax=plt.subplots(figsize=(4,4))
IDXGAVG,IDXRANK=8,12
## scatter and direction
nE1,nI1=np.squeeze(loadingeigvec[IDXGAVG,0,2,:NE,IDXRANK]),np.squeeze(loadingeigvec[IDXGAVG,1,2,:,IDXRANK])
nE2,nI2=np.squeeze(loadingeigvec[IDXGAVG,0,3,:NE,IDXRANK]),np.squeeze(loadingeigvec[IDXGAVG,1,3,:,IDXRANK])
# fig,ax=plt.subplots(figsize=(4,4))
# FIGURE same as overlap
ax.scatter(nE1,nE2,s=5,color='red',alpha=0.5)
ax.scatter(nI1,nI2,s=5,color='blue',alpha=0.5)
for i in range(2):
    nAm2[:,i]=nAm2[:,i]/eigvAm[i]
covnAm=nAm2.T@nAm2
_,eigveccov=la.eig(covnAm)
ax.plot([0,eigveccov[0,1]],[0,eigveccov[1,1]],'pink')